# Kundur Two Areas

### Import Libraries

In [ ]:
import subprocess, sys, os
import urllib.request

dpsim_root_dir = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
sys.path.insert(0, os.path.join(dpsim_root_dir, 'python/src/dpsim/'))
sys.path.insert(0, os.path.join(dpsim_root_dir, 'build'))

import matpower
import dpsimpy
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import urllib.request
import matplotlib.pyplot as plt
import numpy as np
# %matplotlib widget

### Get simulation data

In [ ]:
if not os.path.exists('Kundur2Areas-data'):
    os.mkdir('Kundur2Areas-data')

url_static = 'https://github.com/martinmoraga/dpsim_data/raw/main/Kundur2Areas/matpower/Kundur2Areas.mat'
url_dynamic = 'https://github.com/martinmoraga/dpsim_data/raw/main/Kundur2Areas/matpower/Kundur2Areas_dyn.mat'
local_file_static = './Kundur2Areas-data/Kundur2Areas.mat'
local_file_dynamic = './Kundur2Areas-data/Kundur2Areas_dyn.mat'
urllib.request.urlretrieve(url_static, local_file_static)
urllib.request.urlretrieve(url_dynamic, local_file_dynamic)

### Configure simulation/cosimulation

In [ ]:
time_step = 50e-6
t_f = 1.0
start_fault_time = 0.5
fault_clearing_time = 0.6

switch_closed = 1 # fault resistance
node_fault = "N1"

#  cosimulation parameters
H = 0.4e-3
H_str = '0.4e-3'

assert(time_step < H)

interp = 'linear'
k_map = {'none': 0, 'zoh': 0, 'linear': 1}
k = k_map[interp]

sim_name_dyn_s1 = "EMT_Cosim_Kundur2Areas_dyn_S1" + '_' + interp + '_' + H_str
# subdir_name_s1 = interp + '_' + str(H) + '/'

sim_name_dyn_s2 = "EMT_Cosim_Kundur2Areas_dyn_S2" + '_' + interp + '_' + H_str
# subdir_name_s1 = interp + '_' + str(H) + '/'

cosim_config = {
    "number_topologies": 2,
    "nodes": [["gnd", "N1", "N2", "N5", "N6", "N7", "N8"], ["N3", "N4", "N11", "N10", "N9", "N8"]],
    "eq_component": ["VS", "CS"],
    "split_node": "N8",
    "sim_names": [sim_name_dyn_s1, sim_name_dyn_s2]
}

### 1. Powerflow for initialization

In [ ]:
sim_name_pf = 'Kundur2Areas_PF'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)

# read and create dpsim topology
mpc_reader_pf = matpower.Reader(mpc_file_path=local_file_static, mpc_name='Kundur2Areas')
mpc_reader_pf.load_mpc(domain=matpower.Domain.PF)
system_pf = mpc_reader_pf.system

# log results
logger = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
    logger.log_attribute(node.name()+'.S', 's', node)

# Parametrize and run simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.info)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(1)
sim_pf.set_final_time(0.1)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.add_logger(logger)
sim_pf.run()

### 2. Dynamic, Monolithic simulation

#### Declare some functions

In [ ]:
def Kundur2Areas_dyn(domain="SP"):
    if domain=="SP":
        matpower_domain = matpower.Domain.SP
        dpsim_domain = dpsimpy.Domain.SP
    elif domain=="DP":
        matpower_domain = matpower.Domain.DP
        dpsim_domain = dpsimpy.Domain.DP
    else:
        domain="EMT"
        matpower_domain = matpower.Domain.EMT
        dpsim_domain = dpsimpy.Domain.EMT

    sim_name_dyn = domain + "_Kundur2Areas_dyn"        
    dpsimpy.Logger.set_log_dir('logs/' + sim_name_dyn)

    mpc_reader_dyn = matpower.Reader(mpc_file_path=local_file_static, mpc_name='Kundur2Areas',
                                     mpc_dyn_file_path=local_file_dynamic, mpc_dyn_name='Kundur2Areas_dyn')
    mpc_reader_dyn.create_dpsim_objects(domain=matpower_domain, frequency=60,
                                     with_avr=False, with_tg=False, with_pss=False)

    ### Extend topology with 3ps fault
    sw = mpc_reader_dyn.add_3ph_faut(node_fault, switch_closed=switch_closed, switch_open=1e18)
    
    # create dpsim topology
    mpc_reader_dyn.create_dpsim_topology()

    #initialize node voltages using pf results
    system_dyn = mpc_reader_dyn.system
    system_dyn.init_with_powerflow(system_pf, dpsim_domain)

    # log results
    logger = dpsimpy.Logger(sim_name_dyn)
    for node in system_dyn.nodes:
        logger.log_attribute(node.name()+'.V', 'v', node)

    # Log line currents
    for line_name in ["line3_7-8", "line4_7-8", "line5_8-9", "line6_8-9"]:
        logger.log_attribute('{}.I'.format(line_name), 'i_intf', mpc_reader_dyn.dpsimpy_comp_dict[line_name][0])
        logger.log_attribute('{}.t'.format(line_name), 'ParallelCurrentNode0', mpc_reader_dyn.dpsimpy_comp_dict[line_name][0])
        logger.log_attribute('{}.ParallelCurrentNode1'.format(line_name), 'ParallelCurrentNode1', mpc_reader_dyn.dpsimpy_comp_dict[line_name][0])

    # Parametrize and run simulation
    sim = dpsimpy.Simulation(sim_name_dyn, dpsimpy.LogLevel.info)
    sim.set_system(system_dyn)
    if domain=="SP":
        sim.set_time_step(1e-3)
    else:
        sim.set_time_step(time_step)
    sim.set_final_time(t_f)
    sim.set_domain(dpsim_domain)
    sim.set_solver(dpsimpy.Solver.MNA)
    sim.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.KLU)
    sim.do_init_from_nodes_and_terminals(True)
    sim.add_logger(logger)
    sim.do_system_matrix_recomputation(True)

    # add events
    sw_event_1 = dpsimpy.event.SwitchEvent(start_fault_time, sw, True)
    sw_event_2 = dpsimpy.event.SwitchEvent(fault_clearing_time, sw, False)
    sim.add_event(sw_event_1)
    sim.add_event(sw_event_2)
    
    sim.run()

    return [sim_name_dyn, sim, system_dyn]

#### Dpsim Simulations

In [ ]:
[sim_name_dyn_emt, sim, sys_topo] = Kundur2Areas_dyn(domain="EMT")

#### Get monolithic simulation data

In [ ]:
from villas.dataprocessing.timeseries import TimeSeries as ts
import villas.dataprocessing.plottools as pt
import matplotlib.colors as mcolors

timestep_common = time_step
t_begin = 0.0
t_end = t_f
begin_idx = int(t_begin/timestep_common)
end_idx= int(t_end/timestep_common)
# time = np.linspace(t_begin, t_end, num=end_idx-begin_idx)
time = np.arange(t_begin, t_end, timestep_common)

#plot parameters
width = 8
height = 4

dpsim_result_file_ss = 'logs/' + sim_name_dyn_emt + '/' + sim_name_dyn_emt + '.csv'
ts_dpsim_fault_emt = read_timeseries_csv(dpsim_result_file_ss)

#ts_mono = ts_dpsim_fault_emt["line3_7-8.I_0"].interpolate(timestep_common).time + ts_dpsim_fault_emt["line3_7-8.ParallelCurrentNode1_0"].interpolate(timestep_common).time 
current_3_7_8_0 = ts_dpsim_fault_emt["line3_7-8.I_0"].values
current_4_7_8_0 = ts_dpsim_fault_emt["line4_7-8.I_0"].values

current_parallel0_3_7_8_0 = ts_dpsim_fault_emt["line3_7-8.t_0"].values
current_parallel0_4_7_8_0 = ts_dpsim_fault_emt["line4_7-8.t_0"].values

current_parallel1_3_7_8_0 = ts_dpsim_fault_emt["line3_7-8.ParallelCurrentNode1_0"].values
current_parallel1_4_7_8_0 = ts_dpsim_fault_emt["line4_7-8.ParallelCurrentNode1_0"].values

current_3_7_8_1 = ts_dpsim_fault_emt["line3_7-8.I_1"].values
current_4_7_8_1 = ts_dpsim_fault_emt["line4_7-8.I_1"].values

current_parallel_3_7_8_1 = ts_dpsim_fault_emt["line3_7-8.t_1"].values
current_parallel_4_7_8_1 = ts_dpsim_fault_emt["line4_7-8.t_1"].values

current_3_7_8_2 = ts_dpsim_fault_emt["line3_7-8.I_2"].values
current_4_7_8_2 = ts_dpsim_fault_emt["line4_7-8.I_2"].values

current_parallel_3_7_8_2 = ts_dpsim_fault_emt["line3_7-8.t_2"].values
current_parallel_4_7_8_2 = ts_dpsim_fault_emt["line4_7-8.t_2"].values

current_line_1_0 = current_3_7_8_0 + current_4_7_8_0
# print(current_line_1)
current_parallel_1_0 = current_parallel0_3_7_8_0 + current_parallel0_4_7_8_0
# print(current_parallel_1)

current_line_1_1 = current_3_7_8_1 + current_4_7_8_1
# print(current_line_1)
current_parallel_1_1 = current_parallel_3_7_8_1 + current_parallel_4_7_8_1
# print(current_parallel_1)

current_line_1_2 = current_3_7_8_2 + current_4_7_8_2
# print(current_line_1)
current_parallel_1_2 = current_parallel_3_7_8_2 + current_parallel_4_7_8_2
# print(current_parallel_1)

dpsim_emt_i_line_values_1_0 = current_line_1_0
dpsim_emt_i_intf_values_1_0 = current_line_1_0 - current_parallel_1_0

dpsim_emt_i_line_values_1_1 = current_line_1_1
dpsim_emt_i_intf_values_1_1 = current_line_1_1 - current_parallel_1_1

dpsim_emt_i_line_values_1_2 = current_line_1_2
dpsim_emt_i_intf_values_1_2 = current_line_1_2 - current_parallel_1_2


ts_mono = ts_dpsim_fault_emt["line3_7-8.I_0"].time

current_5_8_9_0 = ts_dpsim_fault_emt["line5_8-9.I_0"].values
current_6_8_9_0 = ts_dpsim_fault_emt["line6_8-9.I_0"].values

current_parallel1_5_8_9_0 = ts_dpsim_fault_emt["line5_8-9.ParallelCurrentNode1_0"].values
current_parallel1_6_8_9_0 = ts_dpsim_fault_emt["line6_8-9.ParallelCurrentNode1_0"].values

current_parallel0_5_8_9_0 = ts_dpsim_fault_emt["line5_8-9.t_0"].values
current_parallel0_6_8_9_0 = ts_dpsim_fault_emt["line6_8-9.t_0"].values

current_5_8_9_1 = ts_dpsim_fault_emt["line5_8-9.I_1"].values
current_6_8_9_1 = ts_dpsim_fault_emt["line6_8-9.I_1"].values

current_parallel_5_8_9_1 = ts_dpsim_fault_emt["line5_8-9.ParallelCurrentNode1_1"].values
current_parallel_6_8_9_1 = ts_dpsim_fault_emt["line6_8-9.ParallelCurrentNode1_1"].values

current_5_8_9_2 = ts_dpsim_fault_emt["line5_8-9.I_2"].values
current_6_8_9_2 = ts_dpsim_fault_emt["line6_8-9.I_2"].values

current_parallel_5_8_9_2 = ts_dpsim_fault_emt["line5_8-9.ParallelCurrentNode1_2"].values
current_parallel_6_8_9_2 = ts_dpsim_fault_emt["line6_8-9.ParallelCurrentNode1_2"].values

current_line_2_0 = current_5_8_9_0 + current_6_8_9_0
current_parallel_2_0 =  current_parallel1_5_8_9_0 + current_parallel1_6_8_9_0

dpsim_emt_i_line_values_2_0 = current_line_2_0
dpsim_emt_i_intf_values_2_0 = current_line_2_0 + current_parallel_2_0

current_line_2_1 = current_5_8_9_1 + current_6_8_9_1
current_parallel_2_1 =  current_parallel_5_8_9_1 + current_parallel_6_8_9_1

dpsim_emt_i_line_values_2_1 = current_line_2_1
dpsim_emt_i_intf_values_2_1 = current_line_2_1 + current_parallel_2_1

current_line_2_2 = current_5_8_9_2 + current_6_8_9_2
current_parallel_2_2 =  current_parallel_5_8_9_2 + current_parallel_6_8_9_2

dpsim_emt_i_line_values_2_2 = current_line_2_2
dpsim_emt_i_intf_values_2_2 = current_line_2_2 + current_parallel_2_2

# Compute current in area 2 to verify the behavior of the parallel
dpsim_emt_i_line_values_2_alt = dpsim_emt_i_intf_values_1_0 - current_parallel_2_0

plt.figure(figsize=(width, height))
#plt.subplot(1, 2, 1)
plt.plot(ts_mono, dpsim_emt_i_intf_values_1_0, label='EMT - Monolithic, i_intf S1')
plt.plot(ts_mono, dpsim_emt_i_line_values_1_0, label='EMT - Monolithic, lines 7-8')
plt.plot(ts_mono, dpsim_emt_i_intf_values_2_0, '--', label='EMT - Monolithic, i_intf S2')
plt.plot(ts_mono, dpsim_emt_i_line_values_2_0, label='EMT - Monolithic, lines 8-9')
plt.plot(ts_mono, dpsim_emt_i_line_values_2_alt, '--', label='EMT - Monolithic, lines 8-9 (calc)')
plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.grid()
# plt.ylim([1400, 1525])
# plt.ylim([-5000, 5000])
# plt.xlim([0, 0.6e-3])
plt.xlabel("time (s)")
plt.ylabel("Interface current")

plt.figure(figsize=(width, height))
#plt.subplot(1, 2, 1)
plt.plot(ts_mono, current_parallel0_3_7_8_0, label='EMT - Monolithic, lines 7-8, current 0')
plt.plot(ts_mono, current_parallel1_3_7_8_0, label='EMT - Monolithic, lines 7-8, current 1')
plt.plot(ts_mono, current_parallel0_5_8_9_0, label='EMT - Monolithic, lines 8-9, current 0')
plt.plot(ts_mono, current_parallel1_5_8_9_0, '--', label='EMT - Monolithic, lines 8-9, current 1')
plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.grid()
# plt.ylim([1400, 1525])
# plt.ylim([-5000, 5000])
# plt.xlim([0, 0.6e-3])
plt.xlabel("time (s)")
plt.ylabel("Parallel current")

In [ ]:
dpsim_emt_v_intf_values_0 = ts_dpsim_fault_emt["N8.V_0"].values
dpsim_emt_v_intf_values_1 = ts_dpsim_fault_emt["N8.V_1"].values
dpsim_emt_v_intf_values_2 = ts_dpsim_fault_emt["N8.V_2"].values

plt.figure(figsize=(width, height))
#plt.subplot(1, 2, 1)
plt.plot(ts_mono, dpsim_emt_v_intf_values_0, label='EMT - Monolithic')
plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.grid()
# plt.ylim([168000, 182000])
# plt.xlim([0, 0.6e-3])
plt.xlabel("time (s)")
plt.ylabel("Interface voltage")

In [ ]:
sys_topo

### Split topologie at node 8

#### Create topologies for cosimulation

In [ ]:
sim_name_fault = "Cosim_Kundur2Areas_Fault"        
dpsimpy.Logger.set_log_dir('logs/' + sim_name_fault)

# load dynamic topology
mpc_reader_dyn = matpower.Reader(mpc_file_path=local_file_static, mpc_name='Kundur2Areas',
                                 mpc_dyn_file_path=local_file_dynamic, mpc_dyn_name='Kundur2Areas_dyn')
mpc_reader_dyn.create_dpsim_objects(domain=matpower.Domain.EMT, frequency=60, 
                                 with_avr=False, with_tg=False, with_pss=False)

### Extend topology with 3ps fault
sw = mpc_reader_dyn.add_3ph_faut(node_fault, switch_closed=switch_closed, switch_open=1e18)

# create dpsim topology
mpc_reader_dyn.create_dpsim_topology()

#initialize node voltages using pf results
system_dyn = mpc_reader_dyn.system
system_dyn.init_with_powerflow(system_pf, dpsimpy.Domain.EMT)

#create topologies for cosimulation
[sys_topo_1, sys_topo_2] = mpc_reader_dyn.create_cosim_topologies(cosim_config)

In [ ]:
sys_topo_1

In [ ]:
sys_topo_2

#### Prepare 2nd sub topology

In [ ]:
dpsimpy.Logger.set_log_dir('logs/' + sim_name_dyn_s2)

init_current_intf_2_rms = sim_pf.get_idobj_attr("line5_8-9", 'current_vector').get()[1][0] + sim_pf.get_idobj_attr("line6_8-9", 'current_vector').get()[1][0]
init_current_parallel_2 = sim.get_idobj_attr("line5_8-9", 'ParallelCurrentNode1').get()[0][0] + sim.get_idobj_attr("line6_8-9", 'ParallelCurrentNode1').get()[0][0]

# set initial reference current of current source
init_current_parallel_2_rms = init_current_parallel_2 * dpsimpy.PEAK1PH_TO_RMS3PH
init_current_line_2 = init_current_intf_2_rms
init_current_line_2_ph3 = dpsimpy.Math.single_phase_variable_to_three_phase(init_current_line_2 * np.sqrt(3))

sys_topo_2.component("CS_N8").set_parameters(init_current_line_2_ph3, 0)

# sys_topo_2.component("CS_N8").set_parameters(dpsimpy.Math.single_phase_variable_to_three_phase(complex(0,0)), 0)

# set parameters of current source
init_voltage_intf_rms = sim_pf.get_idobj_attr("N8", 'v').get()[0][0]
init_voltage_intf = init_voltage_intf_rms * dpsimpy.RMS3PH_TO_PEAK1PH
init_voltage_ph3 = dpsimpy.Math.single_phase_variable_to_three_phase(init_voltage_intf)

sys_topo_2.component("CS_N8").set_intf_voltage(np.real(init_voltage_ph3))

# log results
logger = dpsimpy.Logger(sim_name_dyn_s2)
for node in sys_topo_2.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)

logger.log_attribute(sys_topo_2.component("CS_N8").name()+'.I_ref', 'I_ref', sys_topo_2.component("CS_N8")) 
logger.log_attribute(sys_topo_2.component("CS_N8").name()+'.I', 'i_intf', sys_topo_2.component("CS_N8")) 
logger.log_attribute(sys_topo_2.component("CS_N8").name()+'.V', 'v_intf', sys_topo_2.component("CS_N8"))

# Log line currents
for line_name in ["line5_8-9", "line6_8-9"]:
    logger.log_attribute('{}.I'.format(line_name), 'i_intf', sys_topo_2.component(line_name))

# Parametrize and run simulation
sim2 = dpsimpy.Simulation(sim_name_dyn_s2, dpsimpy.LogLevel.info)
sim2.set_system(sys_topo_2)
sim2.set_time_step(time_step)
sim2.set_final_time(t_f)
sim2.set_domain(dpsimpy.Domain.EMT)
sim2.set_solver(dpsimpy.Solver.MNA)
sim2.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.KLU)
sim2.do_init_from_nodes_and_terminals(True)
sim2.add_logger(logger)
sim2.do_system_matrix_recomputation(True)

if node_fault in cosim_config["nodes"][1]:
    sw_event_1 = dpsimpy.event.SwitchEvent(start_fault_time, sw, True)
    sw_event_2 = dpsimpy.event.SwitchEvent(fault_clearing_time, sw, False)
    sim2.add_event(sw_event_1)
    sim2.add_event(sw_event_2)
   
sim2.start()
y_2_0 = sim2.get_idobj_attr("CS_N8", "v_intf").get()
print(y_2_0)

# Verify initial value of current
init_current_line_2_sim2 = sim2.get_idobj_attr("line5_8-9", "i_intf").get() + sim2.get_idobj_attr("line6_8-9", "i_intf").get()
init_current_parallel_2_sim2 = sim2.get_idobj_attr("line5_8-9", "ParallelCurrentNode1").get() + sim2.get_idobj_attr("line6_8-9", "ParallelCurrentNode1").get()
print(init_current_line_2_sim2 + init_current_parallel_2_sim2)
print(sim2.get_idobj_attr("CS_N8", "i_intf").get())

#### Prepare 1st sub topology

In [ ]:
dpsimpy.Logger.set_log_dir('logs/' + sim_name_dyn_s1)

# set parameters of voltage source
u_1_0 = y_2_0
sys_topo_1.component("VS_N8").set_parameters(V_ref=u_1_0 * dpsimpy.PEAK1PH_TO_RMS3PH , f_src=0)

init_current_intf_1_rms = -(sim_pf.get_idobj_attr("line3_7-8", 'current_vector').get()[0][0] + sim_pf.get_idobj_attr("line4_7-8", 'current_vector').get()[0][0])
init_current_parallel_1 = sim.get_idobj_attr("line3_7-8", 'ParallelCurrentNode0').get()[0][0] + sim.get_idobj_attr("line4_7-8", 'ParallelCurrentNode0').get()[0][0]

# get initial current of voltage source
init_current_intf_1 = init_current_intf_1_rms * dpsimpy.RMS3PH_TO_PEAK1PH
init_current_line_1 = init_current_intf_1
init_current_line_1_ph3 = dpsimpy.Math.single_phase_variable_to_three_phase(init_current_line_1 * np.sqrt(3))

sys_topo_1.component("VS_N8").set_intf_current(np.real(init_current_line_1_ph3))

# log results
logger = dpsimpy.Logger(sim_name_dyn_s1)
for node in sys_topo_1.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)

logger.log_attribute(sys_topo_1.component("VS_N8").name()+'.I', 'i_intf', sys_topo_1.component("VS_N8")) 
logger.log_attribute(sys_topo_1.component("VS_N8").name()+'.V_ref', 'V_ref', sys_topo_1.component("VS_N8"))
logger.log_attribute(sys_topo_1.component("VS_N8").name()+'.V', 'v_intf', sys_topo_1.component("VS_N8"))

for line_name in ["line3_7-8", "line4_7-8"]:
    logger.log_attribute('{}.I'.format(line_name), 'i_intf', sys_topo_1.component(line_name))

# Parametrize and run simulation
sim1 = dpsimpy.Simulation(sim_name_dyn_s1, dpsimpy.LogLevel.info)
sim1.set_system(sys_topo_1)
sim1.set_time_step(time_step)
sim1.set_final_time(t_f)
sim1.set_domain(dpsimpy.Domain.EMT)
sim1.set_solver(dpsimpy.Solver.MNA)
sim1.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.KLU)
sim1.do_init_from_nodes_and_terminals(True)
sim1.add_logger(logger)
sim1.do_system_matrix_recomputation(True)

# add events
if node_fault in cosim_config["nodes"][0]:
    sw_event_1 = dpsimpy.event.SwitchEvent(start_fault_time, sw, True)
    sw_event_2 = dpsimpy.event.SwitchEvent(fault_clearing_time, sw, False)
    sim1.add_event(sw_event_1)
    sim1.add_event(sw_event_2)

sim1.start()

y_1_0 = sim1.get_idobj_attr("VS_N8", "i_intf").get()
# y_1_0 = init_current
print(y_1_0)

# Verify initial value of current
init_current_line_1_sim1 = sim1.get_idobj_attr("line3_7-8", "i_intf").get() + sim1.get_idobj_attr("line4_7-8", "i_intf").get()
init_current_parallel_1_sim1 = sim1.get_idobj_attr("line3_7-8", "ParallelCurrentNode0").get() + sim1.get_idobj_attr("line4_7-8", "ParallelCurrentNode0").get()
print(init_current_line_1_sim1 - init_current_parallel_1_sim1)


#### Execute cosimulation

In [ ]:
# Compare with open-loop results (inputs coming from monolithic)
# dpsim_result_file_dyn_s1_ol = 'logs/' + sim_name_dyn_s1 + '_ol' + '/' + sim_name_dyn_s1 + '.csv'
# ts_dpsim_fault_s1_emt_ol = read_timeseries_csv(dpsim_result_file_dyn_s1_ol)

# dpsim_result_file_dyn_s2_ol = 'logs/' + sim_name_dyn_s2 + '_ol' + '/' + sim_name_dyn_s2 + '.csv'
# ts_dpsim_fault_s2_emt_ol = read_timeseries_csv(dpsim_result_file_dyn_s2_ol)

# dpsim_emt_i_intf_values_s1_ol = ts_dpsim_fault_s1_emt_ol["VS_N8.I_0"].values
# dpsim_emt_i_intf_values_s2_ol = ts_dpsim_fault_s2_emt_ol["CS_N8.I_0"].values
# dpsim_emt_v_intf_values_s1_ol = ts_dpsim_fault_s1_emt_ol["VS_N8.V_0"].values
# dpsim_emt_v_intf_values_s2_ol = ts_dpsim_fault_s2_emt_ol["CS_N8.V_0"].values

t_0 = 0
t_k_v = []
t_k = 0.0

y_1 = y_1_0
y_1_alt = y_1_0

print(y_1)

n = int(round((t_f - t_0) / time_step))
t = np.around(np.linspace(t_0, t_f, n + 1), 16)

m = int(np.around(H/time_step))
print("m = " + str(m))

N = int(round((t_f - t_0) / H))
t_m = np.around(np.linspace(t_0, t_f, N + 1), 16)

# We have to assume the trajectory of y_2 extending its initial value, since we have no prior information
y_1_m_prev = np.tile(y_1, np.min([k+1, m]))

u_2_m_v = np.zeros((len(y_1), n))

if interp == 'none':
    i = 0
    for i in range(0, n):

        # print("y_1 = {}".format(y_1))
        u_2_monolithic = np.array([[dpsim_emt_i_intf_values_1_0[i+1]], [dpsim_emt_i_intf_values_1_1[i+1]], [dpsim_emt_i_intf_values_1_2[i+1]]])
        u_2_monolithic_prev = np.array([[dpsim_emt_i_intf_values_1_0[i]], [dpsim_emt_i_intf_values_1_1[i]], [dpsim_emt_i_intf_values_1_2[i]]])
        
        u_2 = y_1
        u_2_rms = [complex(elem, 0) for elem in u_2 * dpsimpy.PEAK1PH_TO_RMS3PH]
        # print("u_2 = {}".format(u_2))

        # u_2_ol = dpsim_emt_i_intf_values_s1_ol[i]
        
        sim2.get_idobj_attr("CS_N8", "I_ref").set(u_2_rms)
        t_k_2 = sim2.next()
        y_2 = sim2.get_idobj_attr("CS_N8", "v_intf").get()

        u_1_monolithic = np.array([[dpsim_emt_v_intf_values_0[i+1]], [dpsim_emt_v_intf_values_1[i+1]], [dpsim_emt_v_intf_values_2[i+1]]])
        u_1_monolithic_prev = np.array([[dpsim_emt_v_intf_values_0[i]], [dpsim_emt_v_intf_values_1[i]], [dpsim_emt_v_intf_values_2[i]]])
        
        u_1 = y_2
        u_1_rms = [complex(elem, 0) for elem in u_1 * dpsimpy.PEAK1PH_TO_RMS3PH]

        # u_1_ol = dpsim_emt_v_intf_values_s2_ol[i+1]
        
        sim1.get_idobj_attr("VS_N8", "V_ref").set(u_1_rms)
        t_k = sim1.next()
        y_1_prev = y_1
        y_1 = sim1.get_idobj_attr("VS_N8", "i_intf").get()

        # current_line_1_sim1 = sim1.get_idobj_attr("line3_7-8", "i_intf").get() + sim1.get_idobj_attr("line4_7-8", "i_intf").get()
        current_parallel_1_sim1 = sim1.get_idobj_attr("line3_7-8", "ParallelCurrentNode0").get() + sim1.get_idobj_attr("line4_7-8", "ParallelCurrentNode0").get()
        # current_parallel_2_sim2 = sim2.get_idobj_attr("line5_8-9", "ParallelCurrentNode1").get() + sim2.get_idobj_attr("line6_8-9", "ParallelCurrentNode1").get()
        # y_1_alt = current_line_1_sim1 - current_parallel_1_sim1
        # y_1_alt = y_1 - current_parallel_1_sim1
        y_1_alt = y_1_prev

else:

    for i in range(0, N):
        y_1_prev = y_1_m_prev[:,-1]
        t_m_i = t[m*i: m*(i+1)]

        if interp == 'zoh':
            y_1_extrap = np.tile(y_1_prev, (m,1)).T
        elif interp == 'linear':
            t_poly = np.array([i*H-H, i*H])
            f_u_2 = np.polynomial.polynomial.polyfit(t_poly, y_1_m_prev[:,-2:].T, 1)
            y_1_extrap = np.polynomial.polynomial.polyval(t_m_i, f_u_2)
        
        u_2_m = y_1_extrap
        
        u_2_m_v[:,m*i:m*(i+1)] = [np.real(u_2_m_elem) for u_2_m_elem in u_2_m]
        y_1_m = np.zeros((len(y_1_0), m))
        y_2_m = np.zeros((len(y_1_0), m))

        # Switch to S_2
        for j in range(0, m):
            u_2 = u_2_m[:,j]
            
            # print("u_2({}) = {}".format(i, u_2))
            sim2.get_idobj_attr("CS_N8", "I_ref").set([complex(elem, 0) for elem in u_2 * dpsimpy.PEAK1PH_TO_RMS3PH])

            t_k_2 = sim2.next()
            y_2 = sim2.get_idobj_attr("CS_N8", "v_intf").get()
            y_2_m[:,j] = y_2.flatten()

        # Switch to S_1
        u_1_m = y_2_m    
        for j in range(0, m):
            u_1 = u_1_m[:,j]
            
            sim1.get_idobj_attr("VS_N8", "V_ref").set([complex(elem, 0) for elem in u_1 * dpsimpy.PEAK1PH_TO_RMS3PH])
            t_k_1 = sim1.next()
            y_1 = sim1.get_idobj_attr("VS_N8", "i_intf").get()
            # print(y_1)

            y_1_m[:,j] = y_1.flatten()
            t_k_v.append(t_k_1)

        # Option 1: Take values at macrosteps as samples for extrapolation
        # y_1_m_prev = np.hstack((y_1_m_prev, y_1_m[:,-1][np.newaxis].T))

        # Option 2: Take values at microsteps as samples for extrapolation
        y_1_m_prev = y_1_m

        if t_k_1 > t_f:
            print(t_k_1)
            print("i: " + str(i) + ", expected: " + str(N-1))
            print("j: " + str(j) + ", expected: " + str(m-1))
            break

sim1.stop()
sim2.stop()

### Compare steady state results

In [ ]:
dpsim_result_file_dyn_s1 = 'logs/' + sim_name_dyn_s1 + '/' + sim_name_dyn_s1 + '.csv'
ts_dpsim_fault_s1_emt = read_timeseries_csv(dpsim_result_file_dyn_s1)

dpsim_result_file_dyn_s2 = 'logs/' + sim_name_dyn_s2 + '/' + sim_name_dyn_s2 + '.csv'
ts_dpsim_fault_s2_emt = read_timeseries_csv(dpsim_result_file_dyn_s2)

### Define plot functions

In [ ]:
timestep_common_H = H
begin_idx_H = int(t_begin/timestep_common_H)
end_idx_H = int(t_end/timestep_common_H)
# time_H = np.linspace(t_begin, t_end, num=end_idx_H-begin_idx_H)
time_H = np.arange(t_begin, t_end, H)
# print(time_H)

def plot_node_volt_abs(varname_dpsim, 
                       ts_dpsim_emt, ts_dpsim_emt_s1, ts_dpsim_emt_s2, nominal_voltage, timestep_common=1e-3, y_lim=False):
    
    #convert dpsim voltage to magnitude value and per-unit for comparison with psat
    varname_dpsim = varname_dpsim+"_0"  #just plot positive sequence
    dpsim_emt_values_s1 = ts_dpsim_emt_s1[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage
    dpsim_emt_values_s2 = ts_dpsim_emt_s2[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage
    dpsim_emt_values = ts_dpsim_emt[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx] / nominal_voltage
    
    dpsim_emt_values_s1_H = ts_dpsim_emt_s1[varname_dpsim].interpolate(timestep_common_H).values[begin_idx_H:end_idx_H] / nominal_voltage

    plt.figure(figsize=(width, height))
    # plt.subplot(1, 2, 1)
    plt.plot(time, dpsim_emt_values, label='EMT - Monolithic')
    plt.plot(time, dpsim_emt_values_s1, '--v', label='EMT - S1')
    plt.plot(time, dpsim_emt_values_s2, '--v', label='EMT - S2')
    plt.plot(time_H, dpsim_emt_values_s1_H, 'o')
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.grid()
    #plt.ylim([1.058, 1.062])
    #plt.xlim([0.495, 0.62])
    #plt.xlim([0, 10])
    plt.xlabel("time (s)")
    plt.ylabel(varname_dpsim + " (pu)")
    if (y_lim):
        plt.ylim(y_lim)
    
    # plt.subplot(1, 2, 2)
    # plt.plot(time, dpsim_emt_values, ':', label='EMT - DPsim')
    # plt.legend(loc='lower right')
    # plt.xlabel('time (s)')
    # plt.grid()
    # #plt.ylim([1.058, 1.062])
    # # plt.xlim([2.9, 3.1])
    # plt.xlabel("time (s)")
    # plt.ylabel(varname_dpsim + " (pu)")
    # if (y_lim):
    #     plt.ylim(y_lim)
        
    # plt.savefig('Kundur2Areas_dyn_DP_cosim_attributes_fault_{}.svg'.format(varname_dpsim))
    plt.show()
    
    #calculate RMSE
    #rmse = np.sqrt(((dpsim_sp_values_abs_pu - psat_values) ** 2).mean())
    #print('RMSE {:s}  = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(varname_dpsim, rmse, rmse/1.0*100, 1.0))
    
def calculate_3ph_line_power(ts_dpsim_emt, line_name, node_name, timestep_common):
    power_emt = ts_dpsim_emt[line_name+".I_0"].interpolate(timestep_common).values * ts_dpsim_emt[node_name+".V_0"].interpolate(timestep_common).values + \
                ts_dpsim_emt[line_name+".I_1"].interpolate(timestep_common).values * ts_dpsim_emt[node_name+".V_1"].interpolate(timestep_common).values + \
                ts_dpsim_emt[line_name+".I_2"].interpolate(timestep_common).values * ts_dpsim_emt[node_name+".V_2"].interpolate(timestep_common).values
    return power_emt

def plot_power_area12(varname_dpsim, 
                       ts_dpsim_mon, ts_dpsim_s1, ts_dpsim_s2, nominal_voltage, timestep_common=1e-3, y_lim=False):
    
    # calculate power flow throw tie lines
    power_line3_mon = calculate_3ph_line_power(ts_dpsim_mon, "line3_7-8", "N7", timestep_common)
    power_line4_mon = calculate_3ph_line_power(ts_dpsim_mon, "line4_7-8", "N7", timestep_common)
    power_line5_mon = calculate_3ph_line_power(ts_dpsim_mon, "line5_8-9", "N8", timestep_common)
    power_line6_mon = calculate_3ph_line_power(ts_dpsim_mon, "line6_8-9", "N8", timestep_common)

    power_line3_s1 = calculate_3ph_line_power(ts_dpsim_s1, "line3_7-8", "N7", timestep_common)
    power_line4_s1 = calculate_3ph_line_power(ts_dpsim_s1, "line4_7-8", "N7", timestep_common)
    power_line5_s2 = calculate_3ph_line_power(ts_dpsim_s2, "line5_8-9", "N8", timestep_common)
    power_line6_s2 = calculate_3ph_line_power(ts_dpsim_s2, "line6_8-9", "N8", timestep_common)

    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(7, 6))

    axes[0].plot(ts_dpsim_s1["line3_7-8.I_0"].interpolate(timestep_common).time, (power_line3_s1 + power_line4_s1) * 1e-6, label='EMT S1')
    axes[0].plot(ts_dpsim_mon["line3_7-8.I_0"].interpolate(timestep_common).time, (power_line3_mon + power_line4_mon) * 1e-6, '--', label='EMT - Monolithic')
    axes[0].legend(loc='lower right')
    axes[0].set_xlabel('Time [s]')
    axes[0].set_ylabel('Active power flow \nBus 7-8 [MW]')
    #axes[0].set_ylim([0, 10])
    # axes[0].set_xlim([0, 10])
    axes[0].grid()

    axes[1].plot(ts_dpsim_s2["line5_8-9.I_0"].interpolate(timestep_common).time, (power_line5_s2 + power_line6_s2) * 1e-6, label='EMT S2')
    axes[1].plot(ts_dpsim_mon["line4_7-8.I_0"].interpolate(timestep_common).time, (power_line5_mon + power_line6_mon) * 1e-6, '--', label='EMT - Monolithic')
    axes[1].legend(loc='lower right')
    axes[1].set_xlabel('Time [s]')
    axes[1].set_ylabel('Active power flow \nBus 8-9 [MW]')
    # axes[1].set_xlim([0, 10])
    axes[1].grid()

    #fig.tight_layout()
    #fig.savefig("./plotting/ActivePowerExchange_Area1-2.pdf")

#### Inteface node

In [ ]:
# varname_dpsim = 'N8.V'
# # nominal_voltage = 230000
# nominal_voltage = 1
# plot_node_volt_abs(varname_dpsim, ts_dpsim_fault_emt, ts_dpsim_fault_s1_emt, ts_dpsim_fault_s2_emt, nominal_voltage, timestep_common)
# print('H=' + str(H))

In [ ]:
varname_dpsim = 'N8.V'
nominal_voltage = 230000
plot_power_area12(varname_dpsim, ts_dpsim_fault_emt, ts_dpsim_fault_s1_emt, ts_dpsim_fault_s2_emt, nominal_voltage, timestep_common)
plt.savefig('Kundur2Areas_dyn_EMT_cosim_attributes_fault_zoh_0.2e-3_P.svg')

#### Line currents

In [ ]:
ts_s1 = ts_dpsim_fault_s1_emt["VS_N8.I_0"].time
current_3_7_8_s1 = ts_dpsim_fault_s1_emt["line3_7-8.I_0"].values
current_4_7_8_s1 = ts_dpsim_fault_s1_emt["line4_7-8.I_0"].values
current_line_1_s1 = current_3_7_8_s1 + current_4_7_8_s1
dpsim_emt_i_line_1_values_s1 = current_line_1_s1

ts_s2 = ts_dpsim_fault_s2_emt["CS_N8.I_0"].time
current_5_8_9_s2 = ts_dpsim_fault_s2_emt["line5_8-9.I_0"].values
current_6_8_9_s2 = ts_dpsim_fault_s2_emt["line6_8-9.I_0"].values
current_line_2_s2 = current_5_8_9_s2 + current_6_8_9_s2
dpsim_emt_i_line_2_values_s2 = current_line_2_s2

# ts_s1_ol = ts_dpsim_fault_s1_emt_ol["VS_N8.I_0"].time
# ts_s2_ol = ts_dpsim_fault_s2_emt_ol["CS_N8.I_0"].time

plt.figure(figsize=(width, height))
#plt.subplot(1, 2, 1)
plt.plot(ts_mono, dpsim_emt_i_line_values_1_0, label='EMT - Monolithic, lines 7-8')
plt.plot(ts_mono, dpsim_emt_i_line_values_2_0, label='EMT - Monolithic, lines 8-9')
plt.plot(ts_s1, dpsim_emt_i_line_1_values_s1, '--', label='EMT - S1, lines 7-8')
plt.plot(ts_s2, dpsim_emt_i_line_2_values_s2, '--', label='EMT - S2, lines 8-9')
plt.legend(loc='lower left')
plt.xlabel('time (s)')
plt.grid()
# plt.ylim([1400, 1525])
# plt.ylim([-5000, 5000])
# plt.xlim([0, 0.6e-3])
plt.xlabel("time (s)")
plt.ylabel("Interface current")

In [ ]:
#plot parameters
# width = 8
# height = 4

# dpsim_emt_values_s1_H = ts_dpsim_fault_s1_emt["VS_N8.I_0"].interpolate(timestep_common_H).values[begin_idx_H:end_idx_H]

# dpsim_emt_values_s1 = ts_dpsim_fault_s1_emt["line3_7-8.I_0"].interpolate(timestep_common).values + \
#     ts_dpsim_fault_s1_emt["line4_7-8.I_0"].interpolate(timestep_common).values
dpsim_emt_i_intf_values_s1 = ts_dpsim_fault_s1_emt["VS_N8.I_0"].values

# dpsim_emt_values_s2 = ts_dpsim_fault_s2_emt["line5_8-9.I_0"].interpolate(timestep_common).values + \
#     ts_dpsim_fault_s2_emt["line6_8-9.I_0"].interpolate(timestep_common).values
dpsim_emt_i_intf_values_s2 = ts_dpsim_fault_s2_emt["CS_N8.I_0"].values

# dpsim_emt_i_ref_values_s2 = ts_dpsim_fault_s2_emt["CS_N8.I_ref_0"].interpolate(timestep_common).values
# dpsim_emt_values_s2_H = u_2_m_v[0,:]

u_2_m_v_H = u_2_m_v[0,range(begin_idx,end_idx,m)]

# ts_s2 = ts_dpsim_fault_s2_emt["CS_N8.I_0"].interpolate(timestep_common).time
# dpsim_emt_values_s2 = ts_dpsim_fault_s2_emt["CS_N8.I_0"].interpolate(timestep_common).values * dpsimpy.PEAK1PH_TO_RMS3PH
    
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(7, 6))
#plt.subplot(1, 2, 1)
axes[0].plot(ts_mono, dpsim_emt_i_intf_values_1_0, '-', label='EMT - Monolithic, i_intf S1')
# plt.plot(ts_mono, dpsim_emt_i_intf_values_2, '--', label='EMT - Monolithic, i_intf S2')
# plt.plot(time_H, u_2_m_v_H, 'o', label='Macro-step')
axes[0].plot(ts_s1, dpsim_emt_i_intf_values_s1, '--', label='EMT - S1, i_intf')
# axes[0].plot(ts_s1_ol, dpsim_emt_i_intf_values_s1_ol, '--', label='EMT - S1, i_intf, open-loop', color=mcolors.TABLEAU_COLORS['tab:orange'])
axes[0].plot(ts_s2, dpsim_emt_i_intf_values_s2, '-.', label='EMT - S2, i_intf')
# axes[0].plot(ts_s2_ol, dpsim_emt_i_intf_values_s2_ol, '--', label='EMT - S2, i_intf, open-loop', color=mcolors.TABLEAU_COLORS['tab:green'])
# plt.plot(ts_s2, dpsim_emt_i_ref_values_s2, '-.', label='EMT - S2, ref')
# plt.plot(ts_mono[0:-1], u_2_m_v[0,:], ':', label='Linear prediction')
# plt.plot(time_H, dpsim_emt_values_s2_H, 'o', label='Macro-step')

# To show the one-step delay problem
# axes[0].plot(ts_s2[0:-1], dpsim_emt_i_intf_values_s2[1::], '.-', label='EMT - S1, i_intf')

axes[0].legend(loc='lower right')
axes[0].set_xlabel('time (s)')
axes[0].grid()
# plt.ylim([1400, 1525])
# plt.ylim([-5000, 5000])
# plt.xlim([0, 0.6e-3])
# axes[0].set_xlim([0, 0.25e-3])
# axes[0].set_ylim([1466, 1476])
axes[0].set_ylabel("Interface current")

dpsim_emt_v_intf_values_s1 = ts_dpsim_fault_s1_emt["VS_N8.V_0"].values
dpsim_emt_v_ref_values_s1 = ts_dpsim_fault_s1_emt["VS_N8.V_ref_0"].values
dpsim_emt_v_intf_values_s2 = ts_dpsim_fault_s2_emt["CS_N8.V_0"].values

#plt.subplot(1, 2, 1)
axes[1].plot(ts_mono, dpsim_emt_v_intf_values_0, label='EMT - Monolithic')
axes[1].plot(ts_s1, dpsim_emt_v_intf_values_s1, '--', label='EMT - S1')
# axes[1].plot(ts_s1_ol, dpsim_emt_v_intf_values_s1_ol, '--', label='EMT - S1', color=mcolors.TABLEAU_COLORS['tab:orange'])
axes[1].plot(ts_s2, dpsim_emt_v_intf_values_s2, '-.', label='EMT - S2')
# axes[1].plot(ts_s2_ol, dpsim_emt_v_intf_values_s2_ol, '--', label='EMT - S2', color=mcolors.TABLEAU_COLORS['tab:green'])
# plt.plot(ts_s1, dpsim_emt_v_ref_values_s1, '--', label='EMT - S1, ref')
# plt.plot(time_H, dpsim_emt_values_s2_H, 'o', label='Macro-step')
axes[1].legend(loc='lower right')
axes[1].set_xlabel('time (s)')
axes[1].grid()
# plt.ylim([168000, 182000])
# plt.xlim([0, 0.6e-3])
# axes[1].set_xlim([4e-5, 11e-5])
# axes[1].set_ylim([174980, 175600])
axes[1].set_ylabel("Interface voltage")

# To show the one-step delay problem
# axes[1].plot(ts_s1[0:-1], dpsim_emt_v_intf_values_s1[1::], '--', label='EMT - S1, i_intf')

# plt.savefig('Kundur2Areas_dyn_EMT_cosim_attributes_fault_intf.svg')
